In [ ]:
import numpy as np
import cv2
import math

# Your existing data
inv_poly_param = [
    889.91582358,
    528.72413142,
    -20.52846684,
    67.73730056,
    48.02888563,
    -14.14051173,
    20.22746961,
    38.15325087,
    -6.98641569,
    -26.68100659,
    -12.41222046,
    -1.83261524,
]
poly_param = [-5.72041404e02, 0.0, 6.76008376e-04, -3.97814203e-07, 5.05496647e-10]
affine_param = [
    9.99949231e-01,
    -4.44517298e-04,
    6.30993143e-04,
    9.68732891e02,
    7.48514758e02,
]

# Read your image (1920x1536 expected)
img = cv2.imread("./assets/SurCam01.jpeg", cv2.IMREAD_COLOR)
assert img is not None, "Image not found. Check the path."

# Pinhole model: 1280×960, ~90° horizontal FOV
Wp, Hp = 1920, 960
FOV = 120
fx = (Wp / 2.0) / math.tan(math.radians(FOV / 2.0))
pinhole = dict(fx=fx, fy=fx, cx=Wp / 2.0, cy=Hp / 2.0, width=Wp, height=Hp)
fisheye = dict(fx=889.871, fy=889.916, cx=9.68732891e02, cy=7.48514758e02)
print(pinhole)
# Reproject & save
from fisheye_transform import fisheye_to_pinhole

pinhole_img = fisheye_to_pinhole(img, 1.1, pinhole, fisheye)
cv2.imwrite("pinhole_from_fisheye_assumek=1.png", pinhole_img)

In [ ]:
import math
import torch
import torch.nn.functional as F


def fisheye_to_pinhole_torch(
    img_bgr_uint8,  # torch.uint8 [Hf, Wf, 3] 或 [Hf, Wf]
    inv_poly_param,  # list/np.array, len = N (你给的是 12)
    affine_param,  # [c, d, e, xc, yc]
    Wp=1920,
    Hp=960,
    FOV_deg=90.0,  # 目标针孔图尺寸与FOV
):
    """
    将单张鱼眼图（OCamCalib模型）重映射为针孔相机视图（等距FOV裁切）。
    返回 torch.uint8 的针孔图，形状 [Hp, Wp, 3] 或 [Hp, Wp]。
    """
    # ---- 1) 准备输入图像张量 ----
    if img_bgr_uint8.dim() == 2:
        Hf, Wf = img_bgr_uint8.shape
        C = 1
        img = img_bgr_uint8.unsqueeze(0).unsqueeze(0).float()  # [1,1,Hf,Wf]
    else:
        Hf, Wf, _ = img_bgr_uint8.shape
        C = 3
        img = img_bgr_uint8.permute(2, 0, 1).unsqueeze(0).float()  # [1,3,Hf,Wf]

    device = img.device

    # ---- 2) 针孔相机内参（按你的设定）----
    fx = (Wp / 2.0) / math.tan(math.radians(FOV_deg / 2.0))
    fy = fx
    cx = Wp / 2.0
    cy = Hp / 2.0

    # ---- 3) 为目标针孔图生成像素网格 (u,v) ----
    u = torch.linspace(0, Wp - 1, Wp, device=device)
    v = torch.linspace(0, Hp - 1, Hp, device=device)
    uu, vv = torch.meshgrid(u, v, indexing="xy")  # [Hp, Wp]

    # 将像素坐标变成针孔相机坐标系下的方向 (归一化前)
    x = (uu - cx) / fx
    y = (vv - cy) / fy
    z = torch.ones_like(x)

    # 单位化/为 OCamCalib 计算 theta 做准备
    M_norm = torch.sqrt(x * x + y * y)  # sqrt(x^2 + y^2)
    eps = 1e-6
    M_safe = torch.clamp(M_norm, min=eps)
    theta = -torch.atan2(z, M_safe)  # 注意负号，和你给的代码保持一致

    # ---- 4) 计算 rho(theta) = sum a_i * theta^i ----
    inv_poly = torch.tensor(inv_poly_param, dtype=x.dtype, device=device)  # [N]
    # 以 Horner 法则更高效稳定：
    rho = torch.zeros_like(theta)
    for a in reversed(inv_poly):
        rho = rho * theta + a

    # ---- 5) 根据 OCamCalib 仿射把方向映射到鱼眼像素 (u_f, v_f) ----
    c, d, e, xc, yc = [float(t) for t in affine_param]
    # 先把 [x,y] 方向单位化，再乘以 rho
    x_dir = x / M_safe * rho
    y_dir = y / M_safe * rho

    # 2x2 仿射 + 主点
    u_f = c * x_dir + d * y_dir + xc
    v_f = e * x_dir + y_dir + yc

    # ---- 6) grid_sample 采样需要 [-1,1] 归一化坐标 ----
    # 使用 align_corners=True 时： norm = (coord / (size-1)) * 2 - 1
    grid_x = (u_f / (Wf - 1)) * 2.0 - 1.0
    grid_y = (v_f / (Hf - 1)) * 2.0 - 1.0
    grid = torch.stack([grid_x, grid_y], dim=-1).unsqueeze(0)  # [1,Hp,Wp,2]

    # ---- 7) 双线性采样，默认超界使用 0（黑色）----
    pinhole = F.grid_sample(
        img, grid, mode="bilinear", padding_mode="zeros", align_corners=True
    )  # [1,C,Hp,Wp]

    # ---- 8) 输出为 uint8 HxWxC ----
    pinhole = torch.clamp(pinhole, 0, 255).round().byte()
    if C == 1:
        out = pinhole.squeeze(0).squeeze(0)  # [Hp, Wp]
    else:
        out = pinhole.squeeze(0).permute(1, 2, 0)  # [Hp, Wp, 3]
    return out


# ---------------- 使用示例 ----------------
if __name__ == "__main__":
    import cv2
    import numpy as np

    inv_poly_param = [
        889.91582358,
        528.72413142,
        -20.52846684,
        67.73730056,
        48.02888563,
        -14.14051173,
        20.22746961,
        38.15325087,
        -6.98641569,
        -26.68100659,
        -12.41222046,
        -1.83261524,
    ]
    poly_param = [
        -5.72041404e02,
        0.0,
        6.76008376e-04,
        -3.97814203e-07,
        5.05496647e-10,
    ]  # 注：本流程不需要用到
    affine_param = [
        9.99949231e-01,
        -4.44517298e-04,
        6.30993143e-04,
        9.68732891e02,
        7.48514758e02,
    ]

    # 读取 1920x1536 的鱼眼图 (BGR)
    fisheye = cv2.imread("./assets/SurCam01.jpeg", cv2.IMREAD_COLOR)
    assert fisheye is not None and fisheye.shape[1] == 1920 and fisheye.shape[0] == 1536

    # 转 torch
    img_t = torch.from_numpy(fisheye)  # [H,W,3], uint8, BGR

    # 目标分辨率与 FOV
    Wp, Hp, FOV = 1920, 960, 140.0
    pinhole_bgr = (
        fisheye_to_pinhole_torch(
            img_t, inv_poly_param, affine_param, Wp=Wp, Hp=Hp, FOV_deg=FOV
        )
        .cpu()
        .numpy()
    )

    cv2.imwrite(f"rectified_pinhole_1920x960_FOV{FOV}.png", pinhole_bgr)
    print(f"saved -> rectified_pinhole_1920x960_FOV{FOV}.png")